# 1. Import libraries & derive "price_label" & "busiest_day" variables per directions in the reading

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
path = r'/Users/danielmccormick/Desktop/CareerFoundry/Python/08-23 Instacart Basket Analysis'

In [3]:
# importing our cleaned and merged dataframe and running basic data checks on it 
ords_prods_merge = pd.read_pickle(os.path.join(path, 'IC_Data', 'IC_prepared_data', 'orders_products_merged.pkl'))

In [4]:
ords_prods_merge.shape

(32404859, 14)

In [5]:
ords_prods_merge.head()

,order_id,user_id,user_order_count,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both


In [6]:
ords_prods_merge['prices'].value_counts(dropna = False)

12.3    624261
10.3    510496
4.0     447272
6.3     438467
1.3     425691
         ...  
20.2       123
22.7        99
21.5        85
18.3        11
21.0         8
Name: prices, Length: 242, dtype: int64

In [7]:
# designating the data frame to be just the first 1M rows of the imported dataframe
df = ords_prods_merge[:1000000]

In [8]:
# confirming the shape of the dataframe and that is just has 1M rows like we define 
df.shape

(1000000, 14)

In [9]:
df.head()

,order_id,user_id,user_order_count,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both


In [10]:
# defining a function to assign a price label to each item in the dataframe based on its price
def price_label(row):

  if row['prices'] <= 5:
    return 'Low-range product'
  elif (row['prices'] > 5) and (row['prices'] <= 15):
    return 'Mid-range product'
  elif row['prices'] > 15:
    return 'High range'
  else: return 'Not enough data'

In [11]:
# applying the user defined function to the dataframe to create a new column that gives
# which price category the item belongs 
df['price_range'] = df.apply(price_label, axis=1)

/var/folders/3x/syx5xv_d21d0hk0ckl4_nnr40000gn/T/ipykernel_13039/1037845369.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_range'] = df.apply(price_label, axis=1)


In [12]:
# performinga value count to determine the count of items in each price category 
df['price_range'].value_counts(dropna = False)

Mid-range product    756450
Low-range product    243550
Name: price_range, dtype: int64

In [13]:
# confirming there are no items in our 1M row data frame with a price higher than $15 
df['prices'].max()

14.8

In [14]:
# using th df.loc method to peform the same if logic as the user defined function
# if logic is implied by the function and it sorts the results in the same way as the user
# defined function from above 
df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'

/var/folders/3x/syx5xv_d21d0hk0ckl4_nnr40000gn/T/ipykernel_13039/3221315664.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'


In [15]:
df.loc[(df['prices'] <= 15) & (df['prices'] > 5), 'price_range_loc'] = 'Mid-range product'

In [16]:
df.loc[df['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [17]:
# performing a value count to confirm that the df.loc method results in the same counts
# for each price category 
df['price_range_loc'].value_counts(dropna = False)

Mid-range product    756450
Low-range product    243550
Name: price_range_loc, dtype: int64

In [18]:
# using the same df.loc method that we used above on the 1M row dataframe on our full instacart
# dataframe. This will create a new column in the full total dataframe with the price
# categories for each row 
ords_prods_merge.loc[ords_prods_merge['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [19]:
ords_prods_merge.loc[(ords_prods_merge['prices'] <= 15) & (ords_prods_merge['prices'] > 5), 'price_range_loc'] = 'Mid-range product'

In [20]:
ords_prods_merge.loc[ords_prods_merge['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [21]:
# value count to confirm how many items there are in each category for the total data set 
price_range_table = ords_prods_merge['price_range_loc'].value_counts(dropna = False)

Mid-range product     21860860
Low-range product     10126321
High-range product      417678
Name: price_range_loc, dtype: int64

In [22]:
# Copying the frequency count of the difference price_ranges for further analysis in Excel
price_range_table = ords_prods_merge['price_range_loc'].value_counts(dropna = False)
price_range_table.to_clipboard()

In [24]:
# performing a value count to determine how many orders occurred on each day of the week in
# in the total data set 
ords_prods_merge['orders_day_of_week'].value_counts(dropna = False)

0    6204182
1    5660230
6    4496490
2    4213830
5    4205791
3    3840534
4    3783802
Name: orders_day_of_week, dtype: int64

In [25]:
# Copying the order day of the week frequency table to Excel for further analysis
dow_frequency = ords_prods_merge['orders_day_of_week'].value_counts(dropna = False)
dow_frequency.to_clipboard()

In [26]:
# creating an empty list called result. Then creating a for loop based on the results of the 
# value count function from the previous step. We will be appending a new value to 
# the result list for each row in the dataframe. If the order day of the week is 0 (the busiest day)
# then we will append "Busiest Day" to the list, if it is day of the week 4 then we will append
# "Least busy", and for all other days we will append "Regularly Busy"
result = []

for value in ords_prods_merge["orders_day_of_week"]:
  if value == 0:
    result.append("Busiest day")
  elif value == 4:
    result.append("Least busy")
  else:
    result.append("Regularly busy")

In [27]:
# printing the result list we created in the previous step to confirm that values were appened
# for each row in the dataframe 
result

['Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Busiest day',
 'Regularly busy',
 'Reg

In [28]:
# Creating a new column in our dataframe called "busiest_day" and setting the column equal to
# the results from the result list we created in our for loop. In that way each row will have 
# a value for the column that aligns with the day of the week for the order in that row 
ords_prods_merge['busiest_day'] = result

In [29]:
# performing a value count on the newly creating busiest_day row to check how many days fall
# into each category. We can also confirm that the busiest day value count ties exactly to 
# the value count for day 0 in the order day of the week value count above. Also the least busy
# day column count aligns with the count for day 4 above. 
ords_prods_merge['busiest_day'].value_counts(dropna = False)

Regularly busy    22416875
Busiest day        6204182
Least busy         3783802
Name: busiest_day, dtype: int64

In [30]:
# copying the frequency table of the busiest_day flag to Excel for further analysis
busiest_frequency = ords_prods_merge['busiest_day'].value_counts(dropna = False)
busiest_frequency.to_clipboard()

In [31]:
ords_prods_merge.head()

,order_id,user_id,user_order_count,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy


# 2. Deriving "Busiest Days" column with two busiest days and two least busy days

In [32]:
# Creating a list, results_days, and a for loop that will append a value to the list for each 
# row. If the order day of the week is either 0 or 1 (the two busiest days) then "Busier Days"
# will be appended to the list. If the order day of the week is 3 or 4 then "Less Busy Days"
# will be appened. Other wise "Normal busyness days" will be appended to the results_days list
results_days = []

for value in ords_prods_merge['orders_day_of_week']:
   if value in [0,1]:
    results_days.append("Busier Days")
   elif value in [3,4]:
    results_days.append("Less Busy Days")
   else:
    results_days.append("Normal Busyness Days")


In [33]:
# confirming that it appears the values were appended correctly to the results_days list 
results_days

['Normal Busyness Days',
 'Less Busy Days',
 'Less Busy Days',
 'Less Busy Days',
 'Less Busy Days',
 'Normal Busyness Days',
 'Busier Days',
 'Busier Days',
 'Busier Days',
 'Less Busy Days',
 'Busier Days',
 'Normal Busyness Days',
 'Normal Busyness Days',
 'Busier Days',
 'Busier Days',
 'Normal Busyness Days',
 'Normal Busyness Days',
 'Less Busy Days',
 'Less Busy Days',
 'Less Busy Days',
 'Less Busy Days',
 'Less Busy Days',
 'Less Busy Days',
 'Busier Days',
 'Busier Days',
 'Busier Days',
 'Normal Busyness Days',
 'Normal Busyness Days',
 'Busier Days',
 'Normal Busyness Days',
 'Normal Busyness Days',
 'Busier Days',
 'Normal Busyness Days',
 'Normal Busyness Days',
 'Normal Busyness Days',
 'Less Busy Days',
 'Normal Busyness Days',
 'Less Busy Days',
 'Busier Days',
 'Busier Days',
 'Normal Busyness Days',
 'Less Busy Days',
 'Less Busy Days',
 'Normal Busyness Days',
 'Normal Busyness Days',
 'Busier Days',
 'Busier Days',
 'Normal Busyness Days',
 'Busier Days',
 'Busier 

In [34]:
# creating a new column in our ords_prods_merge dataframe called "busiest_days" and appending
# the results_days list values into that column to create a new column that provides the updated
# business category for each row 
ords_prods_merge['busiest_days'] = results_days

In [35]:
ords_prods_merge.head()

,order_id,user_id,user_order_count,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Normal Busyness Days
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Less Busy Days
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Less Busy Days
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Less Busy Days
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Less Busy Days


# 3. Checking new "busiest_days" column for accuracy

In [34]:
# Performing a value count on the newly create busiest_days column to confirm the totals 
# align with the value_counts for each order day of the week from earlier in the exercise.
# After checking the "Normal Busyness Days" count aligns with the sum of order day of the week 
# counts for 2, 5 and 6. The "Busier Days" count aligns with the sum of the order day of the week
# counts for 0 and 1. The "Less Busy Days" count aligns with the sum of the order day of the week
# counts for 3 and 4. 
ords_prods_merge['busiest_days'].value_counts(dropna = False)

Normal Busyness Days    12916111
Busier Days             11864412
Less Busy Days           7624336
Name: busiest_days, dtype: int64

In [37]:
# copying the frequency table of the busiest_days column to Excel for further analysis
busiestdays_frequency = ords_prods_merge['busiest_days'].value_counts(dropna = False)
busiestdays_frequency.to_clipboard()

# 4. Creating a column called "busiest_period_of_day" to break out orders by when they occur

In [35]:
# Creating a column called "busiest_period_of_day" that will group orders into three groups
# based on frequency of orders in that hour. The top four hours for orders will be "Most orders", 
# the bottom four hours for orders will be "Fewest Orders", and the remaining hours will be 
# "Average Orders"

In [36]:
ords_prods_merge['order_hour_of_day'].value_counts(dropna = False)

10    2761760
11    2736140
14    2689136
15    2662144
13    2660954
12    2618532
16    2535202
9     2454203
17    2087654
8     1718118
18    1636502
19    1258305
20     976156
7      891054
21     795637
22     634225
23     402316
6      290493
0      218769
1      115700
5       87961
2       69375
4       53242
3       51281
Name: order_hour_of_day, dtype: int64

In [38]:
# copying the value_counts of order hour of the day to excel for further analysis
hour_frequency = ords_prods_merge['order_hour_of_day'].value_counts(dropna = False)
hour_frequency.to_clipboard()

In [39]:
results_hours = []

for value in ords_prods_merge['order_hour_of_day']:
   if value in [10,11,14,15]:
    results_hours.append("Most Orders")
   elif value in [2,3,4,5]:
    results_hours.append("Fewest Orders")
   else:
    results_hours.append("Average Orders")


In [42]:
results_hours

['Average Orders',
 'Average Orders',
 'Average Orders',
 'Average Orders',
 'Most Orders',
 'Average Orders',
 'Average Orders',
 'Most Orders',
 'Average Orders',
 'Average Orders',
 'Average Orders',
 'Average Orders',
 'Most Orders',
 'Average Orders',
 'Most Orders',
 'Most Orders',
 'Average Orders',
 'Average Orders',
 'Most Orders',
 'Average Orders',
 'Most Orders',
 'Average Orders',
 'Average Orders',
 'Average Orders',
 'Most Orders',
 'Most Orders',
 'Most Orders',
 'Average Orders',
 'Most Orders',
 'Most Orders',
 'Average Orders',
 'Average Orders',
 'Most Orders',
 'Most Orders',
 'Most Orders',
 'Average Orders',
 'Most Orders',
 'Most Orders',
 'Most Orders',
 'Average Orders',
 'Average Orders',
 'Most Orders',
 'Average Orders',
 'Most Orders',
 'Most Orders',
 'Most Orders',
 'Most Orders',
 'Average Orders',
 'Most Orders',
 'Average Orders',
 'Most Orders',
 'Average Orders',
 'Most Orders',
 'Average Orders',
 'Most Orders',
 'Most Orders',
 'Average Orders',
 

In [43]:
# adding the values from our results_hours list to a new column in the dataframe "busiest_period_of_day"
ords_prods_merge['busiest_period_of_day'] = results_hours

# 5. Printing the frequency of the busiest_period_of_day column

In [44]:
# Frequency of each value for the newly created column that classifies the order based 
# on the hour of the day it was made. These values align with the value_count performed above
# for each hour of the day. 
ords_prods_merge['busiest_period_of_day'].value_counts(dropna = False)

Average Orders    21293820
Most Orders       10849180
Fewest Orders       261859
Name: busiest_period_of_day, dtype: int64

In [48]:
# copying busiest hours frequency table to Excel for further analysis
busiest_period_frequency = ords_prods_merge['busiest_period_of_day'].value_counts(dropna = False)
busiest_period_frequency.to_clipboard()

In [45]:
ords_prods_merge.head()

,order_id,user_id,user_order_count,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Normal Busyness Days,Average Orders
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Less Busy Days,Average Orders
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Less Busy Days,Average Orders
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Less Busy Days,Average Orders
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Less Busy Days,Most Orders


# 7. Exporting the dataframe with newly derived variables

In [42]:
ords_prods_merge.to_pickle(os.path.join(path, 'IC_Data', 'IC_prepared_data', 'orders_products_updated.pkl'))